<a href="https://colab.research.google.com/github/it21023996/machine-learning/blob/main/Sinhala_Comments_sentiment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/Singlish Dataset.csv')

In [ ]:
df.head()

,Comments,Sentiment
0,me price dala thiyana widiha hoidai wadi wisth...,Pos
1,ekata nam matath aduna chuti nangiyo oya ayyat...,Neu
2,hoda wada sathanak,Pos
3,ela ela aiya api oyata hamadamath adare,Pos
4,hikvision hodai cctv walata owa nam melo sasar...,Neg


In [ ]:
df.Sentiment.value_counts()

,count
Sentiment,
Pos,4263
Neg,1853
Neu,1716


In [ ]:
print(df.isnull().sum())

Comments     0
Sentiment    0
dtype: int64


In [ ]:
import re

In [ ]:
def clean_text(text):
    text = re.sub(r"[^අ-ෆa-zA-Z\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip().lower()

In [ ]:
df['cleaned'] = df['Comments'].astype(str).apply(clean_text)
df[['Comments', 'cleaned']].head()

,Comments,cleaned
0,me price dala thiyana widiha hoidai wadi wisth...,me price dala thiyana widiha hoidai wadi wisth...
1,ekata nam matath aduna chuti nangiyo oya ayyat...,ekata nam matath aduna chuti nangiyo oya ayyat...
2,hoda wada sathanak,hoda wada sathanak
3,ela ela aiya api oyata hamadamath adare,ela ela aiya api oyata hamadamath adare
4,hikvision hodai cctv walata owa nam melo sasar...,hikvision hodai cctv walata owa nam melo sasar...


In [ ]:
df['tokens'] = df['cleaned'].apply(lambda x: x.split())
df[['cleaned', 'tokens']].head()

,cleaned,tokens
0,me price dala thiyana widiha hoidai wadi wisth...,"[me, price, dala, thiyana, widiha, hoidai, wad..."
1,ekata nam matath aduna chuti nangiyo oya ayyat...,"[ekata, nam, matath, aduna, chuti, nangiyo, oy..."
2,hoda wada sathanak,"[hoda, wada, sathanak]"
3,ela ela aiya api oyata hamadamath adare,"[ela, ela, aiya, api, oyata, hamadamath, adare]"
4,hikvision hodai cctv walata owa nam melo sasar...,"[hikvision, hodai, cctv, walata, owa, nam, mel..."


In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.0
    Uninstalling scipy-1.16.0:
      Successfully uninstalled scipy-1.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
openc

In [ ]:
from gensim.models import FastText

In [ ]:
#turns into list from tokens
corpus = df['tokens'].tolist()

In [ ]:
fasttext_model = FastText(
    sentences=corpus,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    sg=1,
    epochs=10
)

In [ ]:
import numpy as np

In [ ]:
def get_sentence_vector(words, model, vector_size=100):
    vecs = [model.wv[word] for word in words if word in model.wv]
    if len(vecs) == 0:
        return np.zeros(vector_size)
    return np.mean(vecs, axis=0)

In [ ]:
df['vector'] = df['tokens'].apply(lambda x: get_sentence_vector(x, fasttext_model))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['Sentiment'])

In [ ]:
#model needs 2d input to the model .used vstack for that
X = np.vstack(df['vector'].values)
y = df['label_encoded']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
clf = LogisticRegression(max_iter=1000,class_weight='balanced')

In [ ]:
clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

         Neg       0.56      0.73      0.63       371
         Neu       0.46      0.58      0.51       343
         Pos       0.90      0.69      0.78       853

    accuracy                           0.67      1567
   macro avg       0.64      0.66      0.64      1567
weighted avg       0.72      0.67      0.69      1567



SVM

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
svm = LinearSVC(class_weight='balanced', max_iter=10000)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

In [ ]:
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm, target_names=le.classes_))

SVM Classification Report:
              precision    recall  f1-score   support

         Neg       0.57      0.74      0.64       371
         Neu       0.54      0.47      0.50       343
         Pos       0.85      0.78      0.81       853

    accuracy                           0.70      1567
   macro avg       0.65      0.66      0.65      1567
weighted avg       0.71      0.70      0.70      1567



RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [ ]:
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf, target_names=le.classes_))

Random Forest Classification Report:
              precision    recall  f1-score   support

         Neg       0.64      0.66      0.65       371
         Neu       0.54      0.37      0.44       343
         Pos       0.78      0.86      0.82       853

    accuracy                           0.71      1567
   macro avg       0.65      0.63      0.64      1567
weighted avg       0.69      0.71      0.70      1567

